In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Get the list of S&P 500 companies from Wikipedia
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Replace '.' with '-' in ticker symbols for compatibility with Yahoo Finance
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-', regex=False)

# Create a list of tickers (add RSP ETF for comparison)
tickers = sp500['Symbol'].tolist()
tickers.append('RSP')

# Define the start and end date for the data retrieval (last 30 days)
start = (datetime.today() - relativedelta(days=30)).strftime('%Y-%m-%d')
end = datetime.today().strftime('%Y-%m-%d')

# Download the data for all tickers
sp500_data = yf.download(tickers, start=start, end=end)

# Only keep the 'Close' and 'Volume' columns
sp500_data = sp500_data[['Close', 'Volume']]

# Get the most recent closing price and volume for each stock
lastrow_close = sp500_data['Close'].iloc[-1]
lastrow_volume = sp500_data['Volume'].iloc[-1]

# Filter stocks that meet the criteria (price between $10 and $400, volume > 1,000,000)
filtered_tickers = lastrow_close[(lastrow_close > 10) & (lastrow_close < 400) & (lastrow_volume > 999999)].index.tolist()

# Download the data for the filtered tickers
sp500_data = yf.download(filtered_tickers, start=start, end=end)

# Calculate the relative performance of each stock against RSP ETF
relative_performance = sp500_data['Close'].div(sp500_data['Close']['RSP'], axis=0)

# Get the first and last closing prices for each stock to calculate performance
firstrow = relative_performance.iloc[0]
lastrow = relative_performance.iloc[-1]

# Calculate the percentage change in performance
performance = ((lastrow - firstrow) / firstrow * 100).sort_values(ascending=False)

# Create a DataFrame to store the performance data
performance_df = pd.DataFrame(performance, columns=['Performance'])

# Remove the 'RSP' ticker and merge sector information from the original sp500 data
sp500_filtered = sp500.set_index('Symbol').drop('RSP', errors='ignore')
performance_with_sector = performance_df.merge(sp500_filtered[['GICS Sector']], left_index=True, right_index=True, how='left')

# Create a filename with the current date
filename = 'performance_with_sector',datetime.today().strftime('%Y-%m-%d'),'.xlsx'

# Save the results to an Excel file
performance_with_sector.to_excel('performance_with_sector.xlsx', index=True)





[*********************100%***********************]  504 of 504 completed
[*********************100%***********************]  345 of 345 completed


In [2]:
# Display a sample output of
performance_with_sector

,Performance,GICS Sector
Ticker,,
UAL,31.580826,Industrials
PLTR,21.302313,Information Technology
LW,19.514154,Consumer Staples
RJF,19.030120,Financials
NVDA,17.238475,Information Technology
...,...,...
GPC,-17.730901,Consumer Discretionary
MHK,-18.065981,Consumer Discretionary
FSLR,-19.964709,Information Technology
